# data and other import

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

In [ ]:
df=pd.read_csv("gdrive/MyDrive/dataset/heart.csv")
df.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
chestPain=pd.get_dummies(df['cp'],prefix='cp',drop_first=True)
df=pd.concat([df,chestPain],axis=1)
df.drop(['cp'],axis=1,inplace=True)
sp=pd.get_dummies(df['slope'],prefix='slope')
th=pd.get_dummies(df['thal'],prefix='thal')
frames=[df,sp,th]
df=pd.concat(frames,axis=1)
df.drop(['slope','thal'],axis=1,inplace=True)

In [ ]:
df.head(5)

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,ca,target,cp_1,cp_2,cp_3,slope_0,slope_1,slope_2,thal_0,thal_1,thal_2,thal_3
0,63,1,145,233,1,0,150,0,2.3,0,1,0,0,1,1,0,0,0,1,0,0
1,37,1,130,250,0,1,187,0,3.5,0,1,0,1,0,1,0,0,0,0,1,0
2,41,0,130,204,0,0,172,0,1.4,0,1,1,0,0,0,0,1,0,0,1,0
3,56,1,120,236,0,1,178,0,0.8,0,1,1,0,0,0,0,1,0,0,1,0
4,57,0,120,354,0,1,163,1,0.6,0,1,0,0,0,0,0,1,0,0,1,0


# split

In [ ]:
X = df.drop(['target'], axis = 1)
y = df.target.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

# para meter tuning

## lr

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
gs_logiReg=LogisticRegression()

In [ ]:
lr_param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]

In [ ]:
lr_clf = GridSearchCV(gs_logiReg, param_grid = lr_param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
lr_best_clf = lr_clf.fit(X,y)

Fitting 3 folds for each of 1600 candidates, totalling 4800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1402 tasks      | elapsed:   36.6s
[Parallel(n_jobs=-1)]: Done 3114 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 4800 out of 4800 | elapsed:  3.2min finished


In [ ]:
lr_best_clf.best_estimator_

LogisticRegression(C=0.23357214690901212, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
logiReg=LogisticRegression(C=0.23357214690901212, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
logiReg.fit(X_train,y_train)
logiRegPred=logiReg.predict(X_test)
logiRegAcu=accuracy_score(y_test, logiRegPred)
print(logiRegAcu)

0.8852459016393442


## svm classifier

In [ ]:
gs_svm = SVC()

In [ ]:
svm_param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}

In [ ]:
svm_clf = GridSearchCV(gs_svm, param_grid = svm_param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
svm_best_clf = svm_clf.fit(X,y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    0.6s finished


In [ ]:
svm_best_clf.best_estimator_

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
svc = SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
svc.fit(X_train,y_train)
svcPred=svc.predict(X_test)
svcAcu=accuracy_score(y_test, svcPred)
print(svcAcu)

0.8852459016393442


## bayes

## rf

In [ ]:
gs_random_forest = RandomForestClassifier()

In [ ]:
rf_param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
rf_clf = GridSearchCV(gs_random_forest, param_grid = rf_param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
rf_best_clf = rf_clf.fit(X,y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.3min finished


In [ ]:
rf_best_clf.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
randomForest = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=7, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
randomForest.fit(X_train,y_train)
randomForestPred=randomForest.predict(X_test)
randomForestAcu=accuracy_score(randomForestPred,y_test)
print(randomForestAcu)

0.9016393442622951


## DT

In [ ]:
gs_dt = DecisionTreeClassifier()

In [ ]:
dt_param_grid = {
    'criterion': ['entropy', 'gini'],
    'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150],
    'min_samples_split' : range(10,500,20)
    }

In [ ]:
dt_clf = GridSearchCV(gs_dt, param_grid = dt_param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
dt_best_clf = dt_clf.fit(X,y)

Fitting 3 folds for each of 900 candidates, totalling 2700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 956 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 2700 out of 2700 | elapsed:    7.5s finished


In [ ]:
dt_best_clf.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dTree=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=30,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dTree.fit(X_train,y_train)
dTreePred=dTree.predict(X_test)
dTreeAcu=accuracy_score(dTreePred,y_test)
print(dTreeAcu)

0.9016393442622951


## KNN

In [ ]:
gs_knn = KNeighborsClassifier()

In [ ]:
knn_param_grid = {'n_neighbors':np.arange(1,30),
              'leaf_size':np.arange(1,30),
              'algorithm':['auto', 'kd_tree'],
              'n_jobs':[-1]
              }

In [ ]:
knn_clf = GridSearchCV(gs_knn, param_grid = knn_param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
knn_best_clf = knn_clf.fit(X,y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 1682 candidates, totalling 5046 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done 1588 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2488 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 3588 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 4888 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 5046 out of 5046 | elapsed:  4.7min finished


In [ ]:
knn_best_clf.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=13, p=2,
                     weights='uniform')

In [ ]:
knn=KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=13, p=2,
                     weights='uniform')

In [ ]:
knn.fit(X_train,y_train)
knnPred=knn.predict(X_test)
knnAcu=accuracy_score(knnPred,y_test)
print(knnAcu)

0.9016393442622951


## XGBoost

In [ ]:
gs_xgboost = XGBClassifier()
gs_xgboost

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_param_grid = {
    'max_depth': np.arange(1, 10),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05],
    'verbosity': [0,1,2,3],
    "subsample" : [0.5, 0.8]

}

In [ ]:
xgb_clf = GridSearchCV(gs_xgboost, param_grid = xgb_param_grid, cv = 3, verbose=True, n_jobs=-1)

In [ ]:
xgb_best_clf = xgb_clf.fit(X,y)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 1500 tasks      | elapsed:   58.1s
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed:  1.8min finished


In [ ]:
xgb_best_clf.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=0)

In [ ]:
xgboost = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.5, verbosity=0)

In [ ]:
xgboost.fit(X_train, y_train)
xgboost_pred = xgboost.predict(X_test)
xgboost_accu = accuracy_score(y_test, xgboost_pred)
print(xgboost_accu)

0.9180327868852459
